In [1]:
import tensorflow as tf
from tensorflow import keras
import os
tf.__version__

'2.0.0-beta0'

In [11]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.experimental.set_memory_growth(gpus[1], True)

In [12]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [13]:
(img_trn, label_trn), (img_tst, label_tst) = keras.datasets.mnist.load_data()

In [14]:
label_trn.shape

(60000,)

In [15]:
num_pixel = 28*28
img_trn = img_trn.reshape(-1, num_pixel)/255.0
img_tst = img_tst.reshape(-1, num_pixel)/255.0

In [16]:
def create_model():
    model = keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(num_pixel,)),
        keras.layers.Dropout(0.20),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [17]:
model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [18]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)

In [9]:
history = model.fit(img_trn, label_trn, epochs=10, validation_data = (img_tst, label_tst), callbacks=[cp_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
59712/60000 [============================>.] - ETA: 0s - loss: 0.2187 - accuracy: 0.9365
Epoch 00001: saving model to training_1/cp.ckpt
60000/60000 [==============================] - 8s 139us/sample - loss: 0.2183 - accuracy: 0.9367 - val_loss: 0.1082 - val_accuracy: 0.9670
Epoch 2/10
59808/60000 [============================>.] - ETA: 0s - loss: 0.0967 - accuracy: 0.9705
Epoch 00002: saving model to training_1/cp.ckpt
60000/60000 [==============================] - 8s 130us/sample - loss: 0.0966 - accuracy: 0.9706 - val_loss: 0.0815 - val_accuracy: 0.9748
Epoch 3/10
59872/60000 [============================>.] - ETA: 0s - loss: 0.0698 - accuracy: 0.9777
Epoch 00003: saving model to training_1/cp.ckpt
60000/60000 [==============================] - 8s 129us/sample - loss: 0.0697 - accuracy: 0.9777 - val_loss: 0.0804 - val_accuracy: 0.9741
Epoch 4/10
59584/60000 [============================>.] - ETA: 0s - loss: 0.0520 - accura

In [19]:
!ls {checkpoint_dir}

checkpoint
cp.ckpt.data-00000-of-00001
cp.ckpt.index


In [20]:
model = create_model()
loss, acc = model.evaluate(img_tst, label_tst)
print(f'Untrained model; accuracy: {acc*100:5.2f}%')

10000/10000 [==============================] - 1s 88us/sample - loss: 2.3916 - accuracy: 0.0703
Untrained model; accuracy:  7.03%


In [21]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(img_tst, label_tst)
print(f'Restored model; accuracy: {acc*100:5.2f}%')

10000/10000 [==============================] - 0s 46us/sample - loss: 0.0749 - accuracy: 0.9824
Restored model; accuracy: 98.24%


In [22]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True, period=5)

W0607 14:32:17.551599 26484 callbacks.py:859] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


In [18]:
model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(img_trn, label_trn, epochs=50, callbacks=[cp_callback], validation_data = (img_tst, label_tst), verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [23]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [24]:
model.save_weights('./checkpoints/my_checkpoint')
model.load_weights('./checkpoints/my_checkpoint')
loss, acc = model.evaluate(img_tst, label_tst)
print(f'Restored model; accuracy: {acc*100:5.2f}%')

10000/10000 [==============================] - 0s 42us/sample - loss: 0.0749 - accuracy: 0.9824
Restored model; accuracy: 98.24%


In [25]:
model = create_model()
model.load_weights(latest)
model.save('my_model.h5')

In [26]:
model2 = keras.models.load_model('my_model.h5')

W0607 14:32:29.801257 26484 deprecation.py:323] From C:\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0607 14:32:30.048257 26484 util.py:244] Unresolved object in checkpoint: (root).optimizer.iter
W0607 14:32:30.049259 26484 util.py:244] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0607 14:32:30.049259 26484 util.py:244] Unresolved object in checkpoint: (root).optimizer.decay
W0607 14:32:30.050258 26484 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_1
W0607 14:32:30.050258 26484 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_2
W0607 14:32:30.050258 26484 util.py:244] Unresolved object in checkpoint: (root).optimizer.epsilon
W0607 14:32:30.051259 26484 util.py:252] A checkpoint wa

In [27]:
loss, acc = model2.evaluate(img_tst, label_tst)
print(f'Restored model; accuracy: {acc*100:5.2f}%')

10000/10000 [==============================] - 0s 50us/sample - loss: 0.1250 - accuracy: 0.9832
Restored model; accuracy: 98.32%
